In [ ]:
# Chrome ve ChromeDriver'ı yüklemek
!apt-get update # Paketleri güncelle
!apt-get install -y wget curl unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install

# ChromeDriver'ı yüklemek
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

# Selenium ve WebDriver'ı yüklemek
!pip install selenium
!pip install webdriver-manager

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,513 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,620 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from google.colab import files
import html
import unicodedata

# Chrome seçeneklerini ayarlanır
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriver'ı başlatılır
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

yorum_seti = set()  # Benzersiz yorumları saklamak için set kullanılır

# Karakterleri normalleştiren fonksiyon
def normalize_text(text):
    text = html.unescape(text)  # HTML karakterlerini çözer
    text = unicodedata.normalize("NFKD", text)  # Unicode normalizasyonu
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    return text.encode("utf-8", "ignore").decode("utf-8", "ignore")  # UTF-8'e dönüştürüp hataları yok sayılır

for page_num in range(1, 400):  # Örneğin ilk 100 sayfa için
    url = f'https://www.trendyol.com/apple/iphone-13-128-gb-siyah-cep-telefonu-apple-turkiye-garantili-p-150058735/yorumlar?page={page_num}'
    driver.get(url)
    time.sleep(2)  # Sayfanın yüklenmesi için bekle

    # Sayfadaki yorumları çekme işlemi
    yorumlar = driver.find_elements(By.CSS_SELECTOR, '.comment-text')

    # Her yorumu benzersizlik kontrolü yaparak ekleme işlemi
    for yorum in yorumlar:
        yorum_text = yorum.text
        if yorum_text not in yorum_seti:
            yorum_seti.add(yorum_text)  # Yalnızca yeni yorumlar eklenir

    # Eğer sayfa başına 30 yorum varsa, döngü 100 sayfa ile 3000  yoruma ulaşabilir
    if len(yorumlar) < 30:  # Sayfadaki yorum sayısı azaldığında döngüyü durdurulabilir
        print("Daha fazla yeni yorum bulunamadı.")
        break

# Yorumları CSV'ye kaydet
csv_path = '/content/trendyol_reviews_4.csv'
with open(csv_path, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Yorum'])
    for yorum in yorum_seti:
        writer.writerow([yorum])

print(f"{len(yorum_seti)} benzersiz yorum çekildi ve kaydedildi.")
files.download(csv_path)

Daha fazla yeni yorum bulunamadı.
1926 benzersiz yorum çekildi ve kaydedildi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>